# Feature Engineering on a Financial Dataset

You are working for a major bank in the Czech Republic and you have been tasked to analyze the transactions of existing customers. The data team has extracted all the tables from their database they think will be useful for you to analyze the dataset. You will need to consolidate the data from those tables into a single DataFrame and create new features in order to get an enriched dataset from which you will be able to perform an in-depth analysis of customers' banking transactions.

You will be using only the following four tables:

- account: The characteristics of a customer's bank account for a given branch
- client: Personal information related to the bank's customers
- disp: A table that links an account to a customer
- trans: A list of all historical transactions by account

>Note: If you want to know more about these tables, you can look at the [data dictionary](https://github.com/Kusainov/czech-banking-fin-analysis/blob/master/Data%20dictionary.pdf) for this dataset.

In [130]:
import pandas as pd
import numpy as np

In [131]:
df_account = pd.read_csv("../Dataset/account.csv", sep=';')
df_account.rename(
    columns={'date':'account_date'}, 
    inplace=True
)
df_account

,account_id,district_id,frequency,account_date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102
...,...,...,...,...
4495,124,55,POPLATEK MESICNE,971228
4496,3958,59,POPLATEK MESICNE,971228
4497,777,30,POPLATEK MESICNE,971228
4498,1573,63,POPLATEK MESICNE,971229


In [132]:
df_account.account_id.nunique()

4500

In [133]:
df_client = pd.read_csv("../Dataset/client.csv", sep=';')
df_client

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5
...,...,...,...
5364,13955,456030,1
5365,13956,430406,1
5366,13968,680413,61
5367,13971,626019,67


In [134]:
df_client.client_id.nunique()

5369

In [135]:
df_disp = pd.read_csv("../Dataset/disp.csv", sep=';')
df_disp.rename(
    columns={'type':'disp_type'}, 
    inplace=True
)
df_disp

,disp_id,client_id,account_id,disp_type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT
...,...,...,...,...
5364,13647,13955,11349,OWNER
5365,13648,13956,11349,DISPONENT
5366,13660,13968,11359,OWNER
5367,13663,13971,11362,OWNER


In [136]:
df_disp.client_id.nunique()

5369

In [137]:
df_disp.account_id.nunique()

4500

In [138]:
# keep only owners
df_disp_owner = df_disp[df_disp['disp_type'] == 'OWNER']
df_disp_owner

,disp_id,client_id,account_id,disp_type
0,1,1,1,OWNER
1,2,2,2,OWNER
3,4,4,3,OWNER
5,6,6,4,OWNER
6,7,7,5,OWNER
...,...,...,...,...
5363,13623,13931,11333,OWNER
5364,13647,13955,11349,OWNER
5366,13660,13968,11359,OWNER
5367,13663,13971,11362,OWNER


In [139]:
df_trans = pd.read_csv("../Dataset/trans.csv", sep=';')
df_trans.rename(
    columns={'type':'trans_type', 'date':'trans_date'}, 
    inplace=True
)
df_trans

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN


In [140]:
df_trans.account_id.nunique()

4500

In [141]:
df_trans.trans_id.nunique()

1056320

### Merge tables

In [142]:
# merge transaction & account
df_trans_account = pd.merge(
    left=df_trans,
    right=df_account,
    how='left',
    left_on='account_id',
    right_on='account_id'
)
df_trans_account

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id,frequency,account_date
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN,49,POPLATEK MESICNE,960920
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN,68,POPLATEK MESICNE,950515
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN,68,POPLATEK MESICNE,960927
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN,70,POPLATEK MESICNE,930909


In [144]:
# merge transaction & disp
df_trans_disp = pd.merge(
    left=df_trans_account,
    right=df_disp_owner,
    how='left',
    left_on='account_id',
    right_on='account_id'
)
df_trans_disp

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id,frequency,account_date,disp_id,client_id,disp_type
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101,2873,2873,OWNER
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101,4601,4601,OWNER
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102,2397,2397,OWNER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN,49,POPLATEK MESICNE,960920,3512,3512,OWNER
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN,68,POPLATEK MESICNE,950515,3545,3545,OWNER
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN,68,POPLATEK MESICNE,960927,3468,3468,OWNER
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN,70,POPLATEK MESICNE,930909,3513,3513,OWNER


In [148]:
# merge transaction & client
df = pd.merge(
    left=df_trans_disp,
    right=df_client,
    how='left',
    left_on='client_id',
    right_on='client_id'
)
df

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id_x,frequency,account_date,disp_id,client_id,disp_type,birth_number,district_id_y
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101,2873,2873,OWNER,755324,16
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER,365111,74
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER,450114,22
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101,4601,4601,OWNER,350402,1
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102,2397,2397,OWNER,180714,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN,49,POPLATEK MESICNE,960920,3512,3512,OWNER,460525,49
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN,68,POPLATEK MESICNE,950515,3545,3545,OWNER,661101,68
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN,68,POPLATEK MESICNE,960927,3468,3468,OWNER,336231,68
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN,70,POPLATEK MESICNE,930909,3513,3513,OWNER,220720,70


In [149]:
# check 'district_id_x' and 'district_id_y' are the same
df[df.district_id_x != df.district_id_y]

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id_x,frequency,account_date,disp_id,client_id,disp_type,birth_number,district_id_y
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER,365111,74
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER,450114,22
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101,4601,4601,OWNER,350402,1
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102,2397,2397,OWNER,180714,46
6,452728,1539,930103,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,1,POPLATEK PO OBRATU,930103,1866,1866,OWNER,425611,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056244,3626254,2894,981231,PRIJEM,NaN,427.0,59943.5,UROK,NaN,NaN,17,POPLATEK MESICNE,930828,3498,3498,OWNER,380401,77
1056248,3625608,2876,981231,PRIJEM,NaN,195.1,42162.1,UROK,NaN,NaN,52,POPLATEK MESICNE,930422,3476,3476,OWNER,685105,60
1056263,3625933,2884,981231,PRIJEM,NaN,428.1,105066.3,UROK,NaN,NaN,1,POPLATEK MESICNE,930208,3486,3486,OWNER,620309,77
1056283,3627510,2932,981231,PRIJEM,NaN,104.5,33974.7,UROK,NaN,NaN,72,POPLATEK MESICNE,930504,3540,3540,OWNER,360804,51


In [150]:
df.rename(
    columns={'district_id_x':'district_id_client', 'district_id_y':'district_id_account'}, 
    inplace=True
)
df

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id_client,frequency,account_date,disp_id,client_id,disp_type,birth_number,district_id_account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101,2873,2873,OWNER,755324,16
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER,365111,74
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER,450114,22
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101,4601,4601,OWNER,350402,1
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102,2397,2397,OWNER,180714,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN,49,POPLATEK MESICNE,960920,3512,3512,OWNER,460525,49
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN,68,POPLATEK MESICNE,950515,3545,3545,OWNER,661101,68
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN,68,POPLATEK MESICNE,960927,3468,3468,OWNER,336231,68
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN,70,POPLATEK MESICNE,930909,3513,3513,OWNER,220720,70


In [151]:
# check duplicates
df.duplicated().sum()

0

### Datetime features

In [152]:
# fix datetime columns
df['account_date'] = pd.to_datetime(df['account_date'], format='%y%m%d')
df['trans_date'] = pd.to_datetime(df['trans_date'], format='%y%m%d')
df[['account_date', 'trans_date']]

,account_date,trans_date
0,1993-01-01,1993-01-01
1,1993-01-01,1993-01-01
2,1993-01-01,1993-01-01
3,1993-01-01,1993-01-01
4,1993-01-02,1993-01-02
...,...,...
1056315,1996-09-20,1998-12-31
1056316,1995-05-15,1998-12-31
1056317,1996-09-27,1998-12-31
1056318,1993-09-09,1998-12-31


In [153]:
# resolve birth_number
# birth_number is YY MM DD if male else YY MM+50 DD
df['sex'] = df['birth_number'].astype(str).apply(lambda x: 'female' if int(x[2:4]) > 50 else 'male')

In [154]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
def get_birth_date(birth_number):
    birth_number = str(birth_number)
    if int(birth_number[2:4]) > 50:
        amended_birth_number = f'{birth_number[0:2]}{str(int(birth_number[2:4])-50).zfill(2)}{birth_number[4:6]}'
    else:
        amended_birth_number = birth_number

    prelim_birth_date = datetime.strptime(amended_birth_number, '%y%m%d')

    if prelim_birth_date.year >= 2000:
        birth_date = prelim_birth_date - relativedelta(years=100)
    else:
        birth_date = prelim_birth_date

    return birth_date

In [155]:
df['birth_date'] = df['birth_number'].apply(get_birth_date)
df['birth_date']

0         1975-03-24
1         1936-01-11
2         1945-01-14
3         1935-04-02
4         1918-07-14
             ...    
1056315   1946-05-25
1056316   1966-11-01
1056317   1933-12-31
1056318   1922-07-20
1056319   1946-12-02
Name: birth_date, Length: 1056320, dtype: datetime64[ns]

In [156]:
df

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id_client,frequency,account_date,disp_id,client_id,disp_type,birth_number,district_id_account,sex,birth_date
0,695247,2378,1993-01-01,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,1993-01-01,2873,2873,OWNER,755324,16,female,1975-03-24
1,171812,576,1993-01-01,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,1993-01-01,692,692,OWNER,365111,74,female,1936-01-11
2,207264,704,1993-01-01,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,1993-01-01,844,844,OWNER,450114,22,male,1945-01-14
3,1117247,3818,1993-01-01,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,1993-01-01,4601,4601,OWNER,350402,1,male,1935-04-02
4,579373,1972,1993-01-02,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,1993-01-02,2397,2397,OWNER,180714,46,male,1918-07-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,1998-12-31,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN,49,POPLATEK MESICNE,1996-09-20,3512,3512,OWNER,460525,49,male,1946-05-25
1056316,3627616,2935,1998-12-31,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN,68,POPLATEK MESICNE,1995-05-15,3545,3545,OWNER,661101,68,male,1966-11-01
1056317,3625403,2869,1998-12-31,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN,68,POPLATEK MESICNE,1996-09-27,3468,3468,OWNER,336231,68,female,1933-12-31
1056318,3626683,2907,1998-12-31,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN,70,POPLATEK MESICNE,1993-09-09,3513,3513,OWNER,220720,70,male,1922-07-20


### Missing values

In [126]:
df.isna().sum()

client_id                   0
birth_number                0
district_id_client          0
disp_id                     0
account_id                  0
disp_type                   0
district_id_account         0
frequency                   0
account_date                0
trans_id                    0
trans_date                  0
trans_type                  0
operation              218311
amount                      0
balance                     0
k_symbol               569817
bank                   926576
account                901033
sex                         0
birth_date                  0
dtype: int64

### New features

In [167]:
# Customer age when account was opened
df['age_at_account_opened'] = df['account_date'] - df['birth_date']
df['age_at_account_opened'] = df['age_at_account_opened'] / np.timedelta64(1, 'Y')
df['age_at_account_opened'] = df['age_at_account_opened'].round().astype('int')
df['age_at_account_opened']

0          18
1          57
2          48
3          58
4          74
           ..
1056315    50
1056316    29
1056317    63
1056318    71
1056319    46
Name: age_at_account_opened, Length: 1056320, dtype: int32